<a href="https://colab.research.google.com/github/Vaishnavig2909/Vaishnavig2909/blob/main/Final_AI_ML_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr
!pip install SpeechRecognition
!pip install moviepy
!pip install googletrans
!pip install torch
!pip install torchvision
!pip install transformers
!pip install pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
  Created whe

In [ ]:
import easyocr
import speech_recognition as sr
from moviepy.editor import VideoFileClip
from transformers import MarianMTModel, MarianTokenizer
import os
from pydub import AudioSegment

  if event.key is 'enter':



**load the MarianMT translation model and tokenizer**

In [ ]:
def load_translation_model(src_lang='hi', tgt_lang='en'):
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

**function to translate text**

In [ ]:
def translate_text(text, src_lang='hi', tgt_lang='en'):
    tokenizer, model = load_translation_model(src_lang, tgt_lang)
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return translated_text

**function to handle OCR for image input**

In [ ]:
def translate_image(image_path, src_lang='hi', tgt_lang='en'):
    reader = easyocr.Reader([src_lang])
    result = reader.readtext(image_path, detail=0, paragraph=True)
    combined_text = " ".join(result)
    translated_text = translate_text(combined_text, src_lang, tgt_lang)
    return translated_text

**function to transcribe and translate audio**

function to convert MP3 to WAV if needed

In [ ]:
def convert_mp3_to_wav(mp3_path):
    wav_path = mp3_path.replace('.mp3', '.wav')
    sound = AudioSegment.from_mp3(mp3_path)
    sound.export(wav_path, format="wav")
    return wav_path

In [ ]:
def translate_audio(audio_path, src_lang='hi', tgt_lang='en'):
    recognizer = sr.Recognizer()
    if audio_path.endswith('.mp3'):
        audio_path = convert_mp3_to_wav(audio_path)
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
        transcript = recognizer.recognize_google(audio, language=src_lang)
        translated_text = translate_text(transcript, src_lang, tgt_lang)
        return translated_text

**function to extract audio from video, transcribe and translate it**

In [ ]:
def translate_video(video_path, src_lang='hi', tgt_lang='en'):
    video = VideoFileClip(video_path)
    audio_path = "extracted_audio.wav"
    video.audio.write_audiofile(audio_path)

    return translate_audio(audio_path, src_lang, tgt_lang)

**example functions to use for input**

In [ ]:
def process_image_input(image_path):
    print("Processing Image Input...")
    translated_text = translate_image(image_path)
    print(f"Translated Text: {translated_text}")

In [ ]:
def process_audio_input(audio_path):
    print("Processing Audio Input...")
    translated_text = translate_audio(audio_path)
    print(f"Translated Text: {translated_text}")

In [ ]:
def process_video_input(video_path):
    print("Processing Video Input...")
    translated_text = translate_video(video_path)
    print(f"Translated Text: {translated_text}")

**function to ask user for input type**

In [ ]:
def get_user_input():
    print("Select input type:")
    print("1. Image")
    print("2. Audio")
    print("3. Video")

    choice = input("Enter your choice (1/2/3): ")

    if choice == '1':
        image_path = input("Enter the path of the image file: ")
        translated_text = translate_image(image_path)
        print(f"Translated Text: {translated_text}")

    elif choice == '2':
        audio_path = input("Enter the path of the audio file: ")
        translated_text = translate_audio(audio_path)
        print(f"Translated Text: {translated_text}")

    elif choice == '3':
        video_path = input("Enter the path of the video file: ")
        translated_text = translate_video(video_path)
        print(f"Translated Text: {translated_text}")

    else:
        print("Invalid choice. Please enter 1, 2, or 3.")

**start the process with user input**

In [ ]:
get_user_input()